In [13]:
import cv2
import numpy as np
from skimage.measure import label, regionprops
import pandas as pd
from skimage import morphology
from matplotlib import pyplot as plt

# Cargar la imagen
img = cv2.imread('id.jpg', cv2.IMREAD_GRAYSCALE)

In [ ]:


# Verificar si la imagen se cargó correctamente
if img is None:
    print("Error: No se pudo cargar la imagen. Verifica la ruta del archivo.")
else:
    # Preprocesamiento de la imagen
    _, img_bin = cv2.threshold(img, 130, 255, cv2.THRESH_BINARY_INV)

    # Aplicar morfología para eliminar ruido
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_CLOSE, kernel)

    # Etiquetar componentes conectados
    labels = label(img_bin, connectivity=2)
    props = regionprops(labels)

    # Filtrar por área: calcular el área promedio y definir umbrales
    areas = [prop.area for prop in props]
    area_promedio = np.mean(areas)
    umbral_min = area_promedio * 0.5  # Puedes ajustar el factor según tu imagen
    umbral_max = area_promedio * 3

# Ordenar los componentes por la posición horizontal de su bounding box
props_sorted = sorted(props, key=lambda prop: prop.bbox[1])  # Propiedad bbox[1] es min_col (columna izquierda)

# Inicializar una lista para almacenar las características
features_list = []

# Procesar cada componente detectado (ahora ordenado de izquierda a derecha)
for prop in props_sorted:
    # Filtrar por área
    if umbral_min <= prop.area <= umbral_max:
        # Obtener el bounding box del dígito
        min_row, min_col, max_row, max_col = prop.bbox
        digit = img_bin[min_row:max_row, min_col:max_col]
        cv2.rectangle(img, (min_col, min_row), (max_col, max_row), (0, 255, 0), 2)

        # Calcular momentos de Hu
        moments = cv2.moments(digit)
        hu_moments = cv2.HuMoments(moments).flatten()

        # Calcular el número de Euler
        euler_number = prop.euler_number

        # Calcular el área del dígito
        area = prop.area

        # Almacenar los features en una lista
        features_list.append([*hu_moments, euler_number, area])

# Crear un DataFrame de pandas con los features
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
columns = [f'HuMoment_{i+1}' for i in range(7)] + ['EulerNumber', 'Area']
df = pd.DataFrame(features_list, columns=columns)

# Guardar en un archivo Excel
df.to_excel('features_numeros.xlsx', index=False)

print("Extracción de características y guardado en Excel completados.")

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
